# Tutorial on CasADi and Opti
For more information on CasADi, check out https://web.casadi.org/docs/

For IPOPT options, check out https://coin-or.github.io/Ipopt/OPTIONS.html

In [1]:
from casadi import *
import matplotlib.pyplot as plt
from matplotlib.animation import *
from IPython.display import HTML

In [2]:
def plotObjective():
    plt.figure()

    delta = 0.025
    xx = np.arange(-1.7, 1.7, delta)
    yy = np.arange(-1.6, 1.6, delta)
    X, Y = np.meshgrid(xx, yy)
    Z = (1-X)**2 + (Y-X**2)**2
    plt.contour(X, Y, Z, levels=100)

    plt.gca().set_xlim([-1.7, 1.7])
    plt.gca().set_ylim([-1.5, 1.5])
    plt.gca().set_aspect('equal')

def createAnimation(opti_f):
    xx_sol = []
    yy_sol = []
    slopes = np.linspace(-2.0, 2.0, 30)
    for slope in slopes:
        x, y = opti_f(slope)
        xx_sol.append(x)
        yy_sol.append(y)
    
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
             np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4)
    constraint, = plt.plot(np.linspace(-2.0, 2.0, 2), slopes[0]*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    solution, = plt.plot(xx_sol[0], yy_sol[0], marker='o', color='k', markersize=10)
    
    def update_frame(i):
        constraint.set_data(np.linspace(-2.0, 2.0, 2), slopes[i]*np.linspace(-2.0, 2.0, 2))
        solution.set_data([float(xx_sol[i])], [float(yy_sol[i])])
        return constraint, solution,
    
    animation1 = FuncAnimation(plt.gcf(), update_frame, frames=range(len(slopes)))
    
    HTML(animation1.to_jshtml())
    return animation1

## Basic CasADi introduction: variables and functions

## Using CasADi's Algorithmic differentation

## Opti example
(see https://web.casadi.org/blog/opti/)

### Basic Rosenbrock example
\begin{align}
\min_{x,u} \quad (1-x)^2 + (y-x^2)^2
\end{align}

In [ ]:

plotObjective()
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add a simple constraint
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\quad 
\end{align}

In [ ]:

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add an inequality constraint also
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\\
y >= px
\end{align}

In [ ]:

opti.solver("ipopt", {"print_time":False}, {"print_level":5, "max_iter":5})

opti.set_value(p, 2.0)

try:
    sol = opti.solve()
    
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
             np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
    plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

except:
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
             np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
    plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    plt.plot(opti.debug.value(x), opti.debug.value(y), 'k', marker='o', markersize=10)

### More advanced options in Opti

Callbacks

In [ ]:

sol = opti.solve()

Create an opti.to_function object to play around with the parameter p

In [ ]:

print(f"Optimal solution: ({sol_x}, {sol_y})")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
plt.plot(sol_x, sol_y, 'k', marker='o', markersize=10)

In [ ]:
animation1 = createAnimation(opti_f)
HTML(animation1.to_jshtml())

Provide initial guess to prevent convergence to local minimum.

In [ ]:


opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})
opti_f = opti.to_function("opti_f", [p], [x, y])


animation2 = createAnimation(opti_f)
HTML(animation2.to_jshtml())